In [1]:
import sys
sys.path.insert(0, '../Utilities/')

import torch
from collections import OrderedDict

from doe_lhs import *
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
# from plotting import newfig, savefig
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import time
from torch.utils.data import TensorDataset, DataLoader

np.random.seed(1234)

In [2]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
device

device(type='cuda')

In [3]:
# the deep neural network
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        
        # parameters
        self.depth = len(layers) - 1
        
        # set up layer order dict
        self.activation = torch.nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [ ]:
# the physics-guided neural network
class PhysicsInformedNN():
    def __init__(self,x_u, y_u, x_f,x_lb,x_ub,X_u_ts ,u_ts , net, device):
        super(PhysicsInformedNN, self).__init__()
        
        self.batch_size = 10000000
        self.device = device
        shuffle = True

        self.train_x_u = torch.tensor(x_u, requires_grad=True).float().to(self.device)
        self.train_y_u = torch.tensor(y_u, requires_grad=True).float().to(self.device)
        self.train_x_f = torch.tensor(x_f, requires_grad=True).float().to(self.device)

        self.train_x_lb = torch.tensor(x_lb, requires_grad=True).float().to(device)
        self.train_x_ub = torch.tensor(x_ub, requires_grad=True).float().to(device) 
            
    
        # deep neural networks
        self.dnn = net

        # Adam optimizer
        self.Adam_optim = torch.optim.Adam(
            self.dnn.parameters(),
            lr = 1e-4,
            betas = (0.9, 0.999)
        )
        # L-BFGS optimizer
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1e-9,
            line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )
        
        self.train_loader = DataLoader(
            list(zip(self.train_x_u,self.train_y_u)), batch_size=self.batch_size, shuffle=shuffle
        )
        
        self.iter = 0
    
    def get_residual(self,X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(self.device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(self.device)

        u = self.net_u(x, t)
        f = self.net_f(x, t, u)
        return u, f
    
    
    def net_u(self, x, t):  
        u = self.dnn(torch.cat([x, t], dim=1))
        return u
    
    def boundary_loss(self):
        x_lb = torch.tensor(self.train_x_lb[:, 0:1], requires_grad=True).float().to(self.device)
        t_lb = torch.tensor(self.train_x_lb[:, 1:2], requires_grad=True).float().to(self.device)

        x_ub = torch.tensor(self.train_x_ub[:, 0:1], requires_grad=True).float().to(self.device)
        t_ub = torch.tensor(self.train_x_ub[:, 1:2], requires_grad=True).float().to(self.device)
        
        y_pred_lb = self.dnn.forward(torch.cat([x_lb, t_lb], dim=1))
        y_pred_ub = self.dnn.forward(torch.cat([x_ub, t_ub], dim=1))
        
        u_lb = y_pred_lb[:,0:1]
        u_ub = y_pred_ub[:,0:1]

        u_lb_x = torch.autograd.grad(
                u_lb, x_lb, 
                grad_outputs=torch.ones_like(u_lb),
                retain_graph=True,
                create_graph=True
            )[0]

        u_ub_x = torch.autograd.grad(
                u_ub, x_ub, 
                grad_outputs=torch.ones_like(u_ub),
                retain_graph=True,
                create_graph=True
            )[0]

        loss = torch.mean((y_pred_lb - y_pred_ub)**2) + \
               torch.mean((u_lb_x - u_ub_x)**2)
        return loss
    
    
    def net_f(self, x, t, u):
        """ The pytorch autograd version of calculating residual """
        
        u_t = torch.autograd.grad(
            u, t, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_x = torch.autograd.grad(
            u, x, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_xx = torch.autograd.grad(
            u_x, x, 
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0]

        f = u_t - 0.0001* u_xx  + 5*u*u*u - 5*u 
        return f
    
    def loss_func(self):
        
        self.optimizer.zero_grad()
        x = self.train_x_u
        y = self.train_y_u

        y_pred, _ = self.get_residual(x)
        _, f_u  = self.get_residual(self.train_x_f)

        loss_u = torch.mean((y - y_pred) ** 2)
        loss_f = torch.mean(f_u ** 2)

        b_loss = self.boundary_loss()

        loss = loss_u + loss_f + b_loss
        loss.backward()

        self.iter += 1
        if self.iter % 100 == 0:
            u_pred, f_pred = model.predict(self.X)
            error_u = np.linalg.norm(self.U-u_pred,2)/np.linalg.norm(self.U,2)
            print('Iter %d, Error u: %e' % (self.iter, error_u)) 
        return loss

    
    def train(self,epoch,X,U):

        self.train_1(epoch,X,U)
        self.train_2()
    
    def train_1(self,epoch,X,U):
        
        self.X = X
        self.U = U

        for e in range(epoch):
            for i, (x, y) in enumerate(self.train_loader):
                self.dnn.train()
                self.optimizer.zero_grad()

                y_pred, _ = self.get_residual(x)
                _, f_u  = self.get_residual(self.train_x_f)

                loss_u = torch.mean((y - y_pred) ** 2)
                loss_f = torch.mean(f_u ** 2)

                b_loss = self.boundary_loss()

                loss = 100 * loss_u + loss_f + b_loss
                loss.backward(retain_graph = True)
                self.Adam_optim.step()

            self.iter += 1
            if self.iter % 100 == 0:
                u_pred, f_pred = model.predict(X)
                error_u = np.linalg.norm(U-u_pred,2)/np.linalg.norm(U,2)
                print('Iter %d, Error u: %e' % (self.iter, error_u)) 


    def train_2(self):
        self.dnn.train()
        self.optimizer.step(self.loss_func)
            
    def predict(self, X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        u = self.net_u(x, t)
        f = self.net_f(x, t, u)
        u = u.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return u, f

In [ ]:
# the physics-guided neural network
class BC_PhysicsInformedNN():
    def __init__(self,x_u, y_u, x_f,x_lb,x_ub,X_u_ts ,u_ts , net, device):
        super(PhysicsInformedNN, self).__init__()
        
        self.batch_size = 10000000
        self.device = device
        shuffle = True

        self.train_x_u = torch.tensor(x_u, requires_grad=True).float().to(self.device)
        self.train_y_u = torch.tensor(y_u, requires_grad=True).float().to(self.device)
        self.train_x_f = torch.tensor(x_f, requires_grad=True).float().to(self.device)

        self.train_x_lb = torch.tensor(x_lb, requires_grad=True).float().to(device)
        self.train_x_ub = torch.tensor(x_ub, requires_grad=True).float().to(device) 
            
    
        # deep neural networks
        self.dnn = net

        # Adam optimizer
        self.Adam_optim = torch.optim.Adam(
            self.dnn.parameters(),
            lr = 1e-4,
            betas = (0.9, 0.999)
        )
        # L-BFGS optimizer
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1e-9,
            line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )
        
        self.train_loader = DataLoader(
            list(zip(self.train_x_u,self.train_y_u)), batch_size=self.batch_size, shuffle=shuffle
        )
        
        self.iter = 0
    
    def get_residual(self,X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(self.device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(self.device)

        u = self.net_u(x, t)
        f = self.net_f(x, t, u)
        return u, f
    
    
    def net_u(self, x, t):  
        u = self.dnn(torch.cat([x, t], dim=1))
        return u
    
    def boundary_loss(self):
        x_lb = torch.tensor(self.train_x_lb[:, 0:1], requires_grad=True).float().to(self.device)
        t_lb = torch.tensor(self.train_x_lb[:, 1:2], requires_grad=True).float().to(self.device)

        x_ub = torch.tensor(self.train_x_ub[:, 0:1], requires_grad=True).float().to(self.device)
        t_ub = torch.tensor(self.train_x_ub[:, 1:2], requires_grad=True).float().to(self.device)
        
        y_pred_lb = self.dnn.forward(torch.cat([x_lb, t_lb], dim=1))
        y_pred_ub = self.dnn.forward(torch.cat([x_ub, t_ub], dim=1))
        
        u_lb = y_pred_lb[:,0:1]
        u_ub = y_pred_ub[:,0:1]

        u_lb_x = torch.autograd.grad(
                u_lb, x_lb, 
                grad_outputs=torch.ones_like(u_lb),
                retain_graph=True,
                create_graph=True
            )[0]

        u_ub_x = torch.autograd.grad(
                u_ub, x_ub, 
                grad_outputs=torch.ones_like(u_ub),
                retain_graph=True,
                create_graph=True
            )[0]

        loss = torch.mean((y_pred_lb - y_pred_ub)**2) + \
               torch.mean((u_lb_x - u_ub_x)**2)
        return loss
    
    
    def net_f(self, x, t, u):
        """ The pytorch autograd version of calculating residual """
        
        u_t = torch.autograd.grad(
            u, t, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_x = torch.autograd.grad(
            u, x, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_xx = torch.autograd.grad(
            u_x, x, 
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0]

        f = u_t - 0.0001* u_xx  + 5*u*u*u - 5*u 
        return f
    
    def loss_func(self):
        
        self.optimizer.zero_grad()
        x = self.train_x_u
        y = self.train_y_u

        y_pred, _ = self.get_residual(x)
        _, f_u  = self.get_residual(self.train_x_f)

        loss_u = torch.mean((y - y_pred) ** 2)
        loss_f = torch.mean(f_u ** 2)

        b_loss = self.boundary_loss()

        loss = loss_u + loss_f + b_loss
        loss.backward()

        self.iter += 1
        if self.iter % 100 == 0:
            u_pred, f_pred = model.predict(self.X)
            error_u = np.linalg.norm(self.U-u_pred,2)/np.linalg.norm(self.U,2)
            print('Iter %d, Error u: %e' % (self.iter, error_u)) 
        return loss

    
    def train(self,epoch,X,U):

        self.train_1(epoch,X,U)
        self.train_2()
    
    def train_1(self,epoch,X,U):
        
        self.X = X
        self.U = U

        for e in range(epoch):
            for i, (x, y) in enumerate(self.train_loader):
                self.dnn.train()
                self.optimizer.zero_grad()

                y_pred, _ = self.get_residual(x)
                _, f_u  = self.get_residual(self.train_x_f)

                loss_u = torch.mean((y - y_pred) ** 2)
                loss_f = torch.mean(f_u ** 2)

                b_loss = self.boundary_loss()

                loss = 100 * loss_u + loss_f + b_loss
                loss.backward(retain_graph = True)
                self.Adam_optim.step()

            self.iter += 1
            if self.iter % 100 == 0:
                u_pred, f_pred = model.predict(X)
                error_u = np.linalg.norm(U-u_pred,2)/np.linalg.norm(U,2)
                print('Iter %d, Error u: %e' % (self.iter, error_u)) 


    def train_2(self):
        self.dnn.train()
        self.optimizer.step(self.loss_func)
            
    def predict(self, X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        u = self.net_u(x, t)
        f = self.net_f(x, t, u)
        u = u.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return u, f

In [11]:


N_u = 200
N_i = 128
N_f = 10000

# N_u = 200
# N_i = 100
# N_f = 20000

layers = [2, 128, 128, 128, 128, 1]

data = scipy.io.loadmat('./AC.mat')

t = data['tt'].flatten()[:,None] # (201,1)
x = data['x'].flatten()[:,None]  # (512,1)
Exact = np.real(data['uu'])

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.T.flatten()[:,None]              

X_star.shape

start  = 0
step = 50
stop = 201
steps_lb = np.arange(0,stop+step,step)
steps_ub = 1 + steps_lb

iterations = 10000
N_f = 20000 
counter = 0

for i in range(0,steps_lb.size-1):
    t1 = steps_lb[i]
    t2 = steps_ub[i+1]
    temp_t = t[:t2,:]
    t = t[t1:t2,:]
    

0
50
100
150
200
